In [6]:
import pandas as pd
import ee
import geemap
import numpy as np
import random
from datetime import datetime



In [3]:
# Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-sushantniraula01')

## Features collection i.e 
1. DEM Features --> Slope, aspect, elevation
2. Rainfall Features -->precipitation, rainfall_7d, rainfall_360d
3. LLUC and NDVI Features --> LULC and NDVI
4. Various foil Features
5. Distance Features --> Dist_Fault, Dist_Road, Dist_River
6. Geological Features --> GeoID

In [4]:

def collect_landslide_features(input_csv,output_csv):
    """
    Collects multiple geospatial features for landslide locations using Google Earth Engine (GEE).
    
    Parameters:
        input_csv (str): Path to the input CSV file with 'latitude', 'longitude', and 'date' columns.
        output_csv (str): Path to save the output CSV file with extracted features.
    """
    

    
    # Load input data
    df_input = pd.read_csv(input_csv)

    # Convert input data to GEE FeatureCollection
    features = df_input.apply(lambda row: ee.Feature(
        ee.Geometry.Point([row['longitude'], row['latitude']]), 
        {'date': row['date']}
    ), axis=1)
    
    landslide_points = ee.FeatureCollection(features.tolist())

    ### 1️⃣ Extract DEM Features ###
    dem = ee.Image("USGS/SRTMGL1_003")
    slope = ee.Terrain.slope(dem)
    aspect = ee.Terrain.aspect(dem)
    terrain = dem.rename('elevation').addBands(slope.rename('slope')).addBands(aspect.rename('aspect'))
    
    def extract_terrain(feature):
        return feature.set(terrain.reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=feature.geometry(),
            scale=30
        ))
    
    landslide_with_terrain = landslide_points.map(extract_terrain)

    ### 2️⃣ Extract Rainfall Features ###
    chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
    
    def extract_rainfall(feature):
        date = ee.Date(feature.get('date'))
        daily_rainfall = chirps.filterDate(date, date.advance(1, 'day')).mean()
        rainfall_7d = chirps.filterDate(date.advance(-7, 'day'), date).sum()
        rainfall_360d = chirps.filterDate(date.advance(-360, 'day'), date).sum()
    
        rain_values = daily_rainfall.addBands(rainfall_7d.rename('rainfall_7d')) \
                                    .addBands(rainfall_360d.rename('rainfall_360d')) \
                                    .reduceRegion(reducer=ee.Reducer.mean(), geometry=feature.geometry(), scale=5000)
    
        return feature.set(rain_values)
    
    landslide_with_rainfall = landslide_with_terrain.map(extract_rainfall)

    ### 3️⃣ Extract LULC & NDVI Features ###
    lulc = ee.Image("ESA/WorldCover/v200/2021").select('Map')
    ndvi_dataset = ee.ImageCollection("MODIS/061/MOD13Q1").filterDate("2021-01-01", "2021-12-31").select('NDVI')
    
    def extract_lulc_ndvi(feature):
        geometry = feature.geometry()
        land_cover = lulc.reduceRegion(reducer=ee.Reducer.mode(), geometry=geometry, scale=10).get('Map')
        ndvi_current = ndvi_dataset.mean().reduceRegion(reducer=ee.Reducer.mean(), geometry=geometry, scale=250).get('NDVI')
    
        return feature.set({'LULC': land_cover, 'NDVI': ndvi_current})
    
    landslide_with_lulc_ndvi = landslide_with_rainfall.map(extract_lulc_ndvi)

    ### 4️⃣ Extract Soil Properties ###
    soil_features = ee.Image("projects/soilgrids-isric/bdod_mean").select(["bdod_0-5cm_mean"]).rename("Bulk_Density")
    soil_features = soil_features.addBands(ee.Image("projects/soilgrids-isric/cec_mean").select(["cec_0-5cm_mean"]).rename("CEC"))
    soil_features = soil_features.addBands(ee.Image("projects/soilgrids-isric/cfvo_mean").select(["cfvo_0-5cm_mean"]).rename("Coarse_Fragments"))
    soil_features = soil_features.addBands(ee.Image("projects/soilgrids-isric/clay_mean").select(["clay_0-5cm_mean"]).rename("Clay_Content"))
    soil_features = soil_features.addBands(ee.Image("projects/soilgrids-isric/sand_mean").select(["sand_0-5cm_mean"]).rename("Sand_Content"))
    soil_features = soil_features.addBands(ee.Image("projects/soilgrids-isric/silt_mean").select(["silt_0-5cm_mean"]).rename("Silt_Content"))
    soil_features = soil_features.addBands(ee.Image("projects/soilgrids-isric/nitrogen_mean").select(["nitrogen_0-5cm_mean"]).rename("Nitrogen"))
    soil_features = soil_features.addBands(ee.Image("projects/soilgrids-isric/phh2o_mean").select(["phh2o_0-5cm_mean"]).rename("Soil_pH"))
    soil_features = soil_features.addBands(ee.Image("projects/soilgrids-isric/soc_mean").select(["soc_0-5cm_mean"]).rename("Soil_Organic_Carbon"))
    
    def extract_soil(feature):
        return feature.set(soil_features.reduceRegion(reducer=ee.Reducer.mean(), geometry=feature.geometry(), scale=250))
    
    landslide_with_soil = landslide_with_lulc_ndvi.map(extract_soil)

    ### 5️⃣ Extract Distance Features ###
    rivers = ee.FeatureCollection("WWF/HydroSHEDS/v1/FreeFlowingRivers").distance(100000).rename("Dist_River")
    roads = ee.FeatureCollection("projects/sat-io/open-datasets/GRIP4/South-East-Asia").distance(100000).rename("Dist_Road")
    faults = ee.FeatureCollection("projects/ee-sushantniraula01/assets/gem_active_faults_harmonized").distance(100000).rename("Dist_Fault")
    
    distance_layers = rivers.addBands(roads).addBands(faults)
    
    def extract_distance(feature):
        return feature.set(distance_layers.reduceRegion(reducer=ee.Reducer.min(), geometry=feature.geometry(), scale=250))
    
    landslide_with_distance = landslide_with_soil.map(extract_distance)

    ### 6️⃣ Extract Geological Features ###
    geology_nepal = ee.FeatureCollection("projects/ee-sushantniraula01/assets/Geology_Nepal")
    geo8apg = ee.FeatureCollection("projects/ee-sushantniraula01/assets/geo8apg")
    
    def extract_geology(feature):
        geometry = feature.geometry()
        geology_nepal_val = ee.Algorithms.If(geology_nepal.filterBounds(geometry).size().gt(0), geology_nepal.filterBounds(geometry).first().get("GEOL_CODE"), "Unknown")
        geology_geo8apg_val = ee.Algorithms.If(geo8apg.filterBounds(geometry).size().gt(0), geo8apg.filterBounds(geometry).first().get("GEO8AG_ID"), "Unknown")
    
        return feature.set({'Geology_Nepal_ID': geology_nepal_val, 'Geology_geo8apg_ID': geology_geo8apg_val})
    
    landslide_with_geology = landslide_with_distance.map(extract_geology)

    ### Convert FeatureCollection to Pandas DataFrame ###
    def fc_to_df(fc):
        features = fc.getInfo()['features']
        data = []
        for f in features:
            props = f['properties']
            props['longitude'], props['latitude'] = f['geometry']['coordinates']
            data.append(props)
        return pd.DataFrame(data)
    
    # Convert extracted features to DataFrame
    df_final = fc_to_df(landslide_with_geology)
    # Save the final dataset
    df_final.to_csv(output_csv, index=False)
    print(f"✅ Final dataset saved to: {output_csv}")




In [ ]:
collect_landslide_features('landslide_points.csv','final_landslide_features.csv')


In [7]:


def generate_non_landslide_points(input_csv="data_cleaned.csv", output_csv="non_landslide_locations.csv"):
    """
    Generates non-landslide points within Nepal, ensuring they are not near known landslides.
    
    Parameters:
        input_csv (str): Path to the landslide dataset (default: "data_cleaned.csv").
        output_csv (str): Path to save the non-landslide points (default: "non_landslide_locations.csv").
    
    Output:
        Saves a CSV file with 551 non-landslide points, each with latitude, longitude, and a random 2022 date.
    """
    # Load landslide data
    df_landslide = pd.read_csv(input_csv)

    # Load Nepal boundary from GEE
    nepal_boundary = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Nepal'))

    # Generate random points (batch processing)
    num_non_landslide = len(df_landslide)  # Target: 551 non-landslide points
    random_points = []

    while len(random_points) < num_non_landslide * 3:  # Generate more than needed
        rand_lon = random.uniform(80.058, 88.201)  # Longitude within Nepal
        rand_lat = random.uniform(26.347, 30.447)  # Latitude within Nepal
        random_points.append(ee.Geometry.Point([rand_lon, rand_lat]))

    # Create a FeatureCollection of random points
    random_fc = ee.FeatureCollection(random_points)

    # Filter only points inside Nepal
    filtered_fc = random_fc.filterBounds(nepal_boundary)

    # Convert filtered points to a list of coordinates
    valid_points = filtered_fc.geometry().coordinates().getInfo()

    # Convert to Pandas DataFrame
    df_non_landslide = pd.DataFrame(valid_points, columns=["longitude", "latitude"])

    # Ensure no points are too close to landslides
    def is_far_from_landslides(lat, lon):
        """Check if a point is far from all landslide points."""
        close_points = df_landslide[
            ((df_landslide["latitude"] - lat).abs() < 0.01) &
            ((df_landslide["longitude"] - lon).abs() < 0.01)
        ]
        return close_points.empty  # True if far from landslides

    df_non_landslide = df_non_landslide[
        df_non_landslide.apply(lambda row: is_far_from_landslides(row["latitude"], row["longitude"]), axis=1)
    ]

    # Select exactly 551 points
    df_non_landslide = df_non_landslide.head(num_non_landslide)

    # Assign a random date in 2022 (random month and day)
    df_non_landslide["date"] = df_non_landslide.apply(lambda _: 
        datetime(2022, random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d'), axis=1
    )

    # Save the new dataset
    df_non_landslide.to_csv(output_csv, index=False)

    print(f"✅ Generated {len(df_non_landslide)} non-landslide points inside Nepal boundary, saved at {output_csv}")

# Example Usage:
# generate_non_landslide_points()


In [ ]:
generate_non_landslide_points()

In [ ]:
## Visualizing Landslide and Non-Landslide Points on Map



# Load landslide and non-landslide data
df_landslides = pd.read_csv("data_cleaned.csv")
df_non_landslides = pd.read_csv("non_landslide_locations.csv")

# Load Nepal boundary from GEE
nepal_boundary = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Nepal'))

# Convert landslide DataFrame to GEE FeatureCollection
landslide_fc = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([row["longitude"], row["latitude"]]), {"type": "landslide"})
    for _, row in df_landslides.iterrows()
])

# Convert non-landslide DataFrame to GEE FeatureCollection
non_landslide_fc = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([row["longitude"], row["latitude"]]), {"type": "non_landslide"})
    for _, row in df_non_landslides.iterrows()
])

# Clip Landslide & Non-Landslide points to Nepal's boundary
landslide_fc = landslide_fc.filterBounds(nepal_boundary)
non_landslide_fc = non_landslide_fc.filterBounds(nepal_boundary)

# Create geemap interactive map
m = geemap.Map(center=[27.5, 85.3], zoom=7)

boundary_Vis={
    "color": "000000",
    "width": 2,
    "lineType": "solid",
    "fillColor": "00000000",
}
# Add Nepal boundary
m.addLayer(nepal_boundary.style(**boundary_Vis), {}, "Nepal Boundary")

# Add landslide points (Red)
m.addLayer(landslide_fc.style(**{"color": "red", "pointSize": 5}), {}, "Landslides")

# Add non-landslide points (Blue)
m.addLayer(non_landslide_fc.style(**{"color": "blue", "pointSize": 5}), {}, "Non-Landslides")

# Display map
m


In [ ]:
collect_landslide_features('non_landslide_locations.csv','non_landslide_features.csv')